<a href="https://colab.research.google.com/github/albordunos/AI_literacy/blob/main/LDA_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic modeling
- это подход к анализу текстов, который позволяет автоматически выявлять скрытые темы в коллекции документов. Этот подход помогает исследователям обнаруживать основные темы, которые присутствуют в текстах, и группировать документы по этим темам.

# Основные шаги для проведения topic modeling:

## 1. Подготовка данных:
Исследователи должны собрать и подготовить текстовую коллекцию для анализа. Это включает в себя очистку текста от шума, удаление стоп-слов и приведение текста к стандартному формату.



In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Phrases  # Импортируем класс Phrases

# Определение своих стоп-слов
custom_stop_words = ['ai', 'intelligence', 'artificial', 'literacy', 'students', 'learning']

# Очистка данных от стоп слов и токенизация
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english') + custom_stop_words)

def preprocess_text(text):
    if isinstance(text, str):
        tokens = word_tokenize(text.lower())
        filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
        return filtered_tokens
    else:
        return []

# Загрузка данных из Excel файла
file_path = '/content/AILIT_SHORT.xlsx'  #загрузить файл и прописать путь к нему
data = pd.read_excel(file_path)

# Применение функции preprocess_text к столбцу 'Abstract'
data['cleaned_abstract'] = data['Abstract'].apply(preprocess_text) #переименовать название столбца "Abstracts"

print(data.head())

## 2. Выбор модели:
Для проведения topic modeling исследователи должны выбрать подходящую модель, такую как Latent Dirichlet Allocation (LDA) или Non-Negative Matrix Factorization (NMF). Каждая модель имеет свои особенности, и выбор модели зависит от конкретной задачи и структуры данных. Про сравнение моделей можно прочитать [тут](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9120935/).

## 3. Обучение модели:
 После выбора модели исследователи должны обучить ее на текстовых данных. Обучение модели topic modeling включает в себя настройку параметров модели и определение числа тем, которые необходимо выделить.


data['cleaned_abstract'] - это список текстовых данных, из которых будут извлекаться биграммы.

min_count=5 означает, что биграммы, которые встречаются менее 5 раз во всем корпусе текстов, будут проигнорированы.

threshold=10 - это порог ассоциации между словами в биграмме. Чем выше этот параметр, тем более "связанными" должны быть слова, чтобы образовывать биграмму.

start = 2:
Это начальное количество тем для проверки. Модель будет проверять от 2 тем и выше. Выбор 2 как начального значения позволяет избежать моделей с одной темой, которые обычно не дают полезной информации.

limit = 10:
Это максимальное количество тем, которое вы хотите проверить.
В данном случае проверка будет проводиться до 10 тем.
Это значение можно изменить в зависимости от объема данных и целей анализа. Например, если у вас много данных, вы можете увеличить это значение, чтобы исследовать большее количество тем.

step = 1:
Это шаг, с которым будет увеличиваться количество тем на каждой итерации.
В данном случае шаг равен 1, что означает, что количество тем будет увеличиваться на 1 на каждой итерации (2, 3, 4, ..., 10).

In [ ]:
# Создание биграмм
bigram_model = Phrases(data['cleaned_abstract'], min_count=5, threshold=10)
data['bigrams'] = data['cleaned_abstract'].apply(lambda tokens: bigram_model[tokens])

In [ ]:
from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel
import pandas as pd
import matplotlib.pyplot as plt

# Создание словаря и корпуса
dictionary = corpora.Dictionary(data['bigrams'])
corpus = [dictionary.doc2bow(text) for text in data['bigrams']]

# Вычисление TF-IDF
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# Функция для вычисления когерентности, perplexity и log likelihood модели LDA
def compute_metrics(dictionary, corpus, texts, limit=8, start=2, step=1):
    coherence_values = []
    perplexity_values = []
    log_likelihood_values = []
    model_list = []
    num_topics_range = []

    for num_topics in range(start, limit, step):
        model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=150)
        model_list.append(model)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

        perplexity_values.append(-model.log_perplexity(corpus))
        log_likelihood_values.append(model.bound(corpus))

        num_topics_range.append(num_topics)

    return model_list, coherence_values, perplexity_values, log_likelihood_values, num_topics_range

# Параметры для подбора количества тем
start = 2
limit = 6  # максимальное количество тем, которое вы хотите проверить
step = 1

if limit < start:
    print("Ошибка: limit должно быть больше или равно start")
    exit()
if step <= 0:
    print("Ошибка: step должно быть больше 0")
    exit()

# Вычисление метрик для различных значений num_topics
model_list, coherence_values, perplexity_values, log_likelihood_values, num_topics_range = compute_metrics(dictionary, corpus_tfidf, data['bigrams'], limit, start, step)

# Создание таблицы с результатами
metrics_df = pd.DataFrame({
    'Num Topics': num_topics_range,
    'Coherence': coherence_values,
    'Perplexity': perplexity_values,
    'Log Likelihood': log_likelihood_values
})
print(metrics_df)

# Отображение результатов
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.plot(num_topics_range, coherence_values)
plt.xlabel("Количество тем")
plt.ylabel("Coherence")
plt.legend(("Coherence"), loc='best')

plt.subplot(1, 3, 2)
plt.plot(num_topics_range, perplexity_values)
plt.xlabel("Количество тем")
plt.ylabel("Perplexity")
plt.legend(("Perplexity"), loc='best')

plt.subplot(1, 3, 3)
plt.plot(num_topics_range, log_likelihood_values)
plt.xlabel("Количество тем")
plt.ylabel("Log Likelihood")
plt.legend(("Log Likelihood"), loc='best')

plt.tight_layout()
plt.show()

# Вывод количества тем с максимальной когерентностью
optimal_num_topics = num_topics_range[coherence_values.index(max(coherence_values))]
print(f'Оптимальное количество тем по когерентности: {optimal_num_topics}')

# Вывод количества тем с максимальной perplexity
optimal_num_topics_perplexity = num_topics_range[perplexity_values.index(max(perplexity_values))]
print(f'Оптимальное количество тем по perplexity: {optimal_num_topics_perplexity}')

# Вывод количества тем с максимальной log likelihood
optimal_num_topics_log_likelihood = num_topics_range[log_likelihood_values.index(max(log_likelihood_values))]
print(f'Оптимальное количество тем по log likelihood: {optimal_num_topics_log_likelihood}')

In [ ]:
from gensim import models


# Параметры для подбора количества тем
num_topics = int(input("Введите количество тем: "))

# Создание модели LDA с заданным количеством тем
lda_model = models.LdaModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=150)


# Функция для получения доминирующей темы для документа
def get_dominant_topic(lda_model, doc_topics):
    return max(doc_topics, key=lambda item: item[1])

# Получение тем для всех документов
doc_topics = lda_model.get_document_topics(corpus_tfidf)

# Вывод результатов
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {}'.format(idx))
    print('Words: {}'.format(topic))

    # Получение документов, относящихся к теме
    doc_topics = lda_model.get_document_topics(corpus_tfidf)
    docs = [doc_id for doc_id, topic_values in enumerate(doc_topics) if max(topic_values, key=lambda x: x[1])[0] == idx]

    # Вывод номеров строк для темы
    print('Документы (строки):', docs)
    print()

## 4. Интерпретация результатов:

После обучения модели исследователи могут проанализировать результаты topic modeling. Это включает в себя изучение выделенных тем, определение ключевых слов и документов, относящихся к каждой теме, а также оценку качества модели.

In [ ]:
!pip install pyLDAvis


In [ ]:
import gensim
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models

# Создание визуализации
vis_data = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis_data)

# Создание визуализации для каждой темы отдельно
vis_data = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, mds='tsne')
pyLDAvis.display(vis_data)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Получение наиболее вероятных слов для каждой темы
topics_words = lda_model.show_topics(formatted=False)

# Создание облака слов для каждой темы
for topic_id, topic in topics_words:
    word_freq = {word: freq for word, freq in topic}

    # Генерация облака слов
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

    # Визуализация облака слов
    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Topic {topic_id}')
    plt.show()